In [1]:
# import package
from datetime import datetime as dtime
from datetime import date,timedelta
from urllib.request import urlopen
import requests
from dateutil import rrule
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import json
import time
import os
from io import StringIO

In [2]:
DataPath = "./Data/"

In [3]:
# 爬取每月股價的目標網站並包裝成函式
def craw_one_month(stock_number,date):
    url = (
        "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+
        date.strftime('%Y%m%d')+
        "&stockNo="+
        str(stock_number)
    )
    data = json.loads(urlopen(url).read())
    return pd.DataFrame(data['data'],columns=data['fields'])

In [4]:
# 根據使用者輸入的日期，以月為單位，重複呼叫爬取月股價的函式
def craw_stock(stock_number, start_month):
    b_month = date(*[int(x) for x in start_month.split('-')])
    now = dtime.now().strftime("%Y-%m-%d")         # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    
    result = pd.DataFrame()
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        result = pd.concat([result,craw_one_month(stock_number,dt)],ignore_index=True)
        time.sleep(2000.0/1000.0);
    
    return result

In [ ]:
# 建立資料夾
def MKDir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)

In [ ]:
# 建立資料夾，搜尋並且存股票資料
def rrule_test(stock_number, start_month):
    MKDir(DataPath+str(stock_number))
    b_month = date(*[int(x) for x in start_month.split('-')])
    print(b_month.strftime('%Y-%m'))
    now = dtime.now().strftime("%Y-%m-%d")         # 取得現在時間
    e_month = date(*[int(x) for x in now.split('-')])
    print(b_month)
    print(e_month)
    
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        print(dt)
        CurrentData = DataPath + str(stock_number) + '/' + dt.strftime('%Y-%m') + '.csv'
        print(CurrentData)
        if os.path.isfile(CurrentData):
            print("檔案存在。")
        else:
            print("檔案不存在。")
            result = pd.DataFrame()
            result = pd.concat([result,craw_one_month(stock_number,dt)],ignore_index=True)
            result.to_csv(CurrentData, encoding='utf_8_sig')
            time.sleep(2000.0/1000.0)
        
        


In [8]:
# 取得證券編號
def get_stock_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    print(df)
    

In [11]:
def create_tw_stock_info_list():
    res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
    df = pd.read_html(res.text)[0]
    return df
#     df.columns = df.iloc[0]
#     df = df.iloc[1:]
#     df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
#     df = df.dropna(how='any')
#     df = df.reset_index(drop=True)
#     new_df = df['有價證券代號及名稱'].str.replace(u'\u3000',' ').str.split(u' ',expand=True)
#     new_df.columns = ['Ticker', 'StockName']
#     new_df['Sector'] = df['產業別']
#     return new_df

In [ ]:
rrule_test(3034, "2020-05-1")

In [ ]:
df = craw_stock(3034,"2020-10-1")

In [ ]:
df.to_csv('./Data/3034.csv', encoding='utf_8_sig')

In [ ]:
MKDir('Data')

In [12]:
list = create_tw_stock_info_list()
print(list)

                   0                    1           2    3     4        5    6
0          有價證券代號及名稱  國際證券辨識號碼(ISIN Code)         上市日  市場別   產業別  CFICode   備註
1                 股票                   股票          股票   股票    股票       股票   股票
2            1101　台泥         TW0001101004  1962/02/09   上市  水泥工業   ESVUFR  NaN
3            1102　亞泥         TW0001102002  1962/06/08   上市  水泥工業   ESVUFR  NaN
4            1103　嘉泥         TW0001103000  1969/11/14   上市  水泥工業   ESVUFR  NaN
...              ...                  ...         ...  ...   ...      ...  ...
19825  01003T　兆豐新光R1         TW00001003T4  2005/12/26   上市   NaN   CBCIXU  NaN
19826  01004T　土銀富邦R2         TW00001004T2  2006/04/13   上市   NaN   CBCIXU  NaN
19827  01007T　兆豐國泰R2         TW00001007T5  2006/10/13   上市   NaN   CBCIXU  NaN
19828  01009T　王道圓滿R1         TW00001009T1  2018/06/21   上市   NaN   CBCIXU  NaN
19829  01010T　京城樂富R1         TW00001010T9  2018/12/05   上市   NaN   CBCIXU  NaN

[19830 rows x 7 columns]


In [1]:
datestr = '20180131'

# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')

# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)

# 整理一些字串：
df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))

# 顯示出來
df.head()

NameError: name 'requests' is not defined

In [27]:
print(df)

         證券代號 證券名稱     成交股數  成交筆數       成交金額    開盤價    最高價    最低價    收盤價  \
0        50.0  NaN  4251739  1176  359040915  84.40  85.00  84.15  84.65   
1        51.0  NaN     7500     3     243040  32.39  32.39  32.39  32.39   
2        52.0  NaN     4001     5     229706  57.35  57.60  57.30  57.40   
3        53.0  NaN        1     1         37    NaN    NaN    NaN    NaN   
4        54.0  NaN     2001     3      48904  24.44  24.44  24.44  24.44   
...       ...  ...      ...   ...        ...    ...    ...    ...    ...   
13718  9944.0  NaN    31013    28     612197  19.75  19.75  19.70  19.70   
13719  9945.0  NaN  3736138  1927  124164872  33.00  33.70  32.65  33.45   
13720  9946.0  NaN   188743    83    2429334  12.80  13.00  12.80  13.00   
13721  9955.0  NaN    69012    49    1037730  14.80  15.15  14.80  15.00   
13722  9958.0  NaN  7951033  3588  337437513  39.10  43.65  39.10  43.65   

       漲跌(+/-)  漲跌價差  最後揭示買價  最後揭示買量  最後揭示賣價  最後揭示賣量     本益比  Unnamed: 16  
0          

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13723 entries, 0 to 13722
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   證券代號         11177 non-null  float64
 1   證券名稱         0 non-null      float64
 2   成交股數         13723 non-null  int64  
 3   成交筆數         13723 non-null  int64  
 4   成交金額         13723 non-null  int64  
 5   開盤價          9401 non-null   float64
 6   最高價          9401 non-null   float64
 7   最低價          9401 non-null   float64
 8   收盤價          9401 non-null   float64
 9   漲跌(+/-)      7714 non-null   float64
 10  漲跌價差         13723 non-null  float64
 11  最後揭示買價       13090 non-null  float64
 12  最後揭示買量       13723 non-null  int64  
 13  最後揭示賣價       13306 non-null  float64
 14  最後揭示賣量       13723 non-null  int64  
 15  本益比          13723 non-null  float64
 16  Unnamed: 16  0 non-null      float64
dtypes: float64(12), int64(5)
memory usage: 1.8 MB


In [24]:
df['證券名稱'].astype('object')
df['證券名稱'] = df['證券名稱'].astype('object')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13723 entries, 0 to 13722
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   證券代號         11177 non-null  float64
 1   證券名稱         0 non-null      object 
 2   成交股數         13723 non-null  int64  
 3   成交筆數         13723 non-null  int64  
 4   成交金額         13723 non-null  int64  
 5   開盤價          9401 non-null   float64
 6   最高價          9401 non-null   float64
 7   最低價          9401 non-null   float64
 8   收盤價          9401 non-null   float64
 9   漲跌(+/-)      7714 non-null   float64
 10  漲跌價差         13723 non-null  float64
 11  最後揭示買價       13090 non-null  float64
 12  最後揭示買量       13723 non-null  int64  
 13  最後揭示賣價       13306 non-null  float64
 14  最後揭示賣量       13723 non-null  int64  
 15  本益比          13723 non-null  float64
 16  Unnamed: 16  0 non-null      float64
dtypes: float64(11), int64(5), object(1)
memory usage: 1.8+ MB


In [29]:
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,50.0,NaN,4251739,1176,359040915,84.40,85.00,84.15,84.65,1.0,0.10,84.60,21,84.65,80,0.0,NaN
1,51.0,NaN,7500,3,243040,32.39,32.39,32.39,32.39,1.0,0.09,32.39,43,32.50,5,0.0,NaN
2,52.0,NaN,4001,5,229706,57.35,57.60,57.30,57.40,1.0,0.05,57.35,1,57.90,1,0.0,NaN
3,53.0,NaN,1,1,37,NaN,NaN,NaN,NaN,NaN,0.00,37.09,50,37.35,1,0.0,NaN
4,54.0,NaN,2001,3,48904,24.44,24.44,24.44,24.44,-1.0,0.05,24.41,50,24.64,8,0.0,NaN
